In [1]:
import gzip
import kipoiseq
from kipoiseq import Interval
import pyfaidx
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle
import sys
import tensorflow as tf
import importlib
import copy
import time


myDir = os.path.join(os.path.dirname(os.getcwd()), "bin")
model_path = "/home/luisasantus/Desktop/crg_cluster/data/FED/enformer/1"
sys.path.append(myDir)
from utils_full import *
model = Enformer(model_path)

from enformer import *


SEQUENCE_LENGHT = 393_216
## pad the sequence with Ns (anyways ignored by the model)
def pad_one_hot(sequence_one_hot, NEW_SIZE):
    ADD_ENDS = int((NEW_SIZE - sequence_one_hot.shape[0])/2)
    pad_zero = np.tile(np.array([0., 0., 0., 0.]), (ADD_ENDS, 1))
    padded_left = np.append(pad_zero,sequence_one_hot, axis=0)
    pad_sequence = np.append(padded_left,pad_zero, axis=0)
    return(pad_sequence)


2022-02-09 16:15:04.688006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-09 16:15:04.688024: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-09 16:15:08.138585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-09 16:15:08.138602: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-09 16:15:08.138616: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (luisasantus-HP-EliteDesk-800-G5-TWR): /proc/driver/nvidia/version does not exist
2022-02-09 16:15:08.138759: I tensorflow/

In [ ]:
import multiprocessing

In [ ]:
dataset_197k_file = "/home/luisasantus/Desktop/crg_cluster/data/FED/basenji/human/tfrecords_197k/valid-0-0_197k.pkl"
with open(dataset_197k_file, 'rb') as file:
    dataset_197k = pickle.load(file) 

In [ ]:
test_ds = dataset_197k[1:2]

In [ ]:
test =  "/home/luisasantus/Desktop/crg_cluster/data/FED/basenji/human/test_197.pkl"
with open(test, 'wb') as file:
    pickle.dump(test_ds, file) 

In [ ]:
def evaluate_model_all_sequences_mod(model, dataset, head, dataset_197k_evaluation, max_steps=None):

    # Given a tensor with a one-encoded sequence, predicts head tracks
    def predict(x):
        padded_sequence = pad_one_hot(np.squeeze(x.numpy(), axis=0), SEQUENCE_LENGHT)[np.newaxis]
        predictions = model.predict_on_batch(padded_sequence)[head]
        return tf.convert_to_tensor(predictions, dtype=tf.float32)

    for i in range(len(dataset)):
        if max_steps is not None and i > max_steps:
            break
            
        batch = dataset[i]
        seq = batch['sequence']
        t0 = time.time()
        prediction = predict(seq)
        t1 = time.time()
        pred_time = t1-t0
        print("predtime "+ str(pred_time))
        metric_seq = MetricDict({'PearsonR': PearsonR(reduce_axis=(0,1))})
        metric_seq.update_state(batch['target'][np.newaxis], prediction)
        t2 = time.time()
        metric_seq_time = t2-t1
        print("metric time  "+ str(metric_seq_time))
        
        pearson_seq = metric_seq.result()["PearsonR"].numpy()
        batch_validation = {"sequence": batch["sequence"],
                            "target": batch["target"],
                            "interval": batch["interval"],
                            "PearsonR": pearson_seq}
        dataset_197k_evaluation.append(batch_validation)
        
        
        t3 = time.time()
        object_time = t3-t2
        print("object time  "+ str(object_time))


    return dataset_197k_evaluation

In [ ]:
dataset_197k[1]

In [ ]:
evaluate_model_all_sequences_mod(model,
                               dataset=test_ds,
                               head="human",
                               dataset_197k_evaluation = [], 
                               max_steps=1)

In [ ]:
def test(test_ds):
    return evaluate_model_all_sequences_mod(model,
                               dataset=test_ds,
                               head="human",
                               dataset_197k_evaluation = [], 
                               max_steps=1)
    

In [ ]:
import multiprocessing as mp

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(2)



In [ ]:
def evaluate_model_seq(model, batch, head, max_steps=None):
     
    prediction = predict(batch['sequence'])
    metric_seq = MetricDict({'PearsonR': PearsonR(reduce_axis=(0,1))})
    metric_seq.update_state(batch['target'][np.newaxis], prediction)        
    pearson_seq = metric_seq.result()["PearsonR"].numpy()
    batch_validation = {"sequence": batch["sequence"],
                            "target": batch["target"],
                            "interval": batch["interval"],
                            "PearsonR": pearson_seq}

    return batch_validation

In [ ]:
a = list([test_ds, test_ds, test_ds, test_ds])

In [ ]:
a

In [ ]:
def arrt(inp):
    return(1)

In [ ]:
def evaluate_model_seq(model, batch, head, max_steps=None):
    
    prediction = predict(batch['sequence'])
    metric_seq = MetricDict({'PearsonR': PearsonR(reduce_axis=(0,1))})
    metric_seq.update_state(batch['target'][np.newaxis], prediction)        
    pearson_seq = metric_seq.result()["PearsonR"].numpy()
    batch_validation = {"sequence": batch["sequence"],
                            "target": batch["target"],
                            "interval": batch["interval"],
                            "PearsonR": pearson_seq}

    return batch_validation

In [ ]:
SEQUENCE_LENGHT

In [ ]:
def evaluate_model_seqxx(batch):
    SEQUENCE_LENGHT = 393216
    
    def predict(x):
        return x
    
    prediction = predict(batch['sequence'])
    

    return prediction

In [ ]:
global model

In [ ]:
global model 

def evaluate_model_4(batch):
    global head 
    prediction = batch['sequence']
    
    return head

In [ ]:
# Step 2: `pool.apply` the `howmany_within_range()`
args = [example ]
results = [pool.apply(evaluate_model_4, args=args) for example in a]


In [ ]:
import tensorflow as tf

raw_dataset = tf.data.TFRecordDataset("/home/luisasantus/Desktop/crg_cluster/data/FED/basenji/human/data_human_tfrecords_valid-0-0.tfr", compression_type='ZLIB')


In [ ]:
shards = 2

for i in range(shards):
    writer = tf.data.experimental.TFRecordWriter(f"output_file-part-{i}.tfrecord")
    writer.write(raw_dataset.shard(shards, i))

In [ ]:
def split_tfrecord(tfrecord_path, split_size):
    with tf.Graph().as_default(), tf.Session() as sess:
        ds = tf.data.TFRecordDataset(tfrecord_path).batch(split_size)
        batch = ds.make_one_shot_iterator().get_next()
        part_num = 0
        while True:
            try:
                records = sess.run(batch)
                part_path = tfrecord_path + '.{:03d}'.format(part_num)
                with tf.python_io.TFRecordWriter(part_path) as writer:
                    for record in records:
                        writer.write(record)
                part_num += 1
            except tf.errors.OutOfRangeError: break

In [ ]:
import tensorflow as tf

ITEMS_PER_FILE = 100 # Assuming we are saving 100 items per .tfrecord file


batch_idx = 0
for batch in raw_dataset.batch(ITEMS_PER_FILE):

    # Converting `batch` back into a `Dataset`, assuming batch is a `tuple` of `tensors`
    batch_ds = tf.data.Dataset.from_tensor_slices(tuple([*batch]))
    filename = f'out.tfrecord.{batch_idx:03d}'

    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(batch_ds)

    batch_idx += 1

In [ ]:

import multiprocessing as mp

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

# Step 2: `pool.apply` the `howmany_within_range()`
results = [pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data]

# Step 3: Don't forget to close
pool.close()    

print(results[:10])
#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
list(chunks(range(10, 75), 10))

In [ ]:
lst = list([1,2,3,4,5,6,7,8])

In [2]:
dataset_197k_file = "/home/luisasantus/Desktop/crg_cluster/data/FED/basenji/human/tfrecords_197k/valid-0-0_197k.pkl"
with open(dataset_197k_file, 'rb') as file:
    dataset_197k = pickle.load(file) 

In [3]:
len(dataset_197k)

256